# Integrais

Nesta seção, encerraremos discutindo **integrais**, que calculam áreas sob uma função. Também chamado de **antiderivada**, o processo é realizado por meio de **integração**. Usaremos novamente o SymPy para nos auxiliar no cálculo de integrais e evitar o trabalho de lápis e papel que você talvez tenha feito no ensino médio e na faculdade.

## Descobrindo a Integração

Digamos que temos esta função.

$ 
\Large f(x) = 2x - 2
$

In [ ]:
from sympy import * 

x = symbols('x')
f = 2*x - 2

plot(f,xlim=(-5,5), ylim=(-5,5)) 

Agora, digamos que você queira encontrar a área sob a função entre $ x = 1 $ e $ x = 3 $, mas acima do eixo x. Vamos sombreá-la passando o parâmetro `fill` abaixo para `plot()`.

In [ ]:
from sympy import * 
import numpy as np 

x = symbols('x')
f = 2*x - 2

x_array = np.linspace(1, 3, 1000)
f_array = lambdify(x, f)(x_array)

plot(f, xlim=(-5,5), ylim=(-5,5), fill={'x': x_array,'y1':f_array, 'alpha': .5}) 

Calcular esta área não é um grande problema. Podemos ver claramente que se trata de um triângulo e podemos usar a fórmula básica da geometria:

$ 
\Large A = \frac{1}{2} bh
$ 

Portanto, se a base é $ 2 $ e a altura é $ 4 $, descobrimos que a área é $ 4 $.


$ 
\Large A = \frac{1}{2} (2)(4) = 4
$ 

Também podemos usar a função `integrate()` no SymPy para calcular a área deste intervalo. Exploraremos essa função mais adiante.

In [ ]:
from sympy import * 

x = symbols('x')
f = 2*x - 2

integrate(f, (x,1,3))

Agora, o que dizer de uma função curvilínea que não seja uma reta? Qual é a área entre $ 0 $ e $ 1 $?

$
\Large f(x) = 2 x^{2} + 1
$

In [ ]:
from sympy import * 
import numpy as np 

x = symbols('x')
f = 2*x**2 + 1

x_array = np.linspace(0, 1, 1000)
f_array = lambdify(x, f)(x_array)

plot(f, xlim=(-2,2), ylim=(-1,5), fill={'x': x_array,'y1':f_array, 'alpha': .5}) 

Opa. Isso não é tão simples, pois não temos mais bordas retas para calcular. Agora, coincidentemente, podemos usar o SymPy para calcular isso usando a função `integrate()` novamente. Obtemos uma área de $ 5/3 $.

In [ ]:
from sympy import * 

x = symbols('x')
f = 2*x**2 + 1

integrate(f, (x,0,1))

Mas como isso funciona? Como podemos encontrar a área sob uma função como essa? Bem, tudo se resume, mais uma vez... a um limite!

## Somas de Riemann

Veja como podemos raciocinar sobre como encontrar a área sob qualquer função dada. Digamos que empacotamos alguns retângulos de largura igual sob a função para o intervalo dado em que estamos interessados. Criaremos uma função `plot_reimann_sums()` que faz exatamente isso, e vamos começar com 5 retângulos.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np 

def plot_reimann_sums(f, eixo_x_inferior, eixo_x_superior, x_inferior, x_superior, n_rects): 
    
    # Define o intervalo para o eixo x
    x_vals = np.linspace(eixo_x_inferior, eixo_x_superior, 1000)
    
    # Calcula os valores da função para os valores do eixo x
    y_vals = f(x_vals)
    
    # Cria o enredo
    fig, ax = plt.subplots()

    # Define os retângulos
    x_rects = np.linspace(x_inferior, x_superior, n_rects+1)
    y_rects = f(x_rects)

    # plota retângulos
    for x,y,next_x in zip(x_rects, y_rects, x_rects[1:]): 
        ax.add_patch(Rectangle((x, 0), next_x-x, y, alpha=.5, color='orange'))

    plt.plot(x_vals, y_vals)
    
    # Mostra o gráfico
    plt.show()

def f(x): return 2*x**2 + 1
    
plot_reimann_sums(f, eixo_x_inferior=-2, eixo_x_superior=2, x_inferior=0, x_superior=1, n_rects=5)

É fácil calcular a área de retângulos, certo? Então, podemos aproximar a área somando a área sob os retângulos. Mas e se usássemos mais retângulos com larguras menores? Como seriam 10 retângulos?

In [ ]:
plot_reimann_sums(f, eixo_x_inferior=-2, eixo_x_superior=2, x_inferior=0, x_superior=1, n_rects=10)

Que tal 20 retângulos?

In [ ]:
plot_reimann_sums(f, eixo_x_inferior=-2, eixo_x_superior=2, x_inferior=0, x_superior=1, n_rects=20)

É justo dizer que quanto mais retângulos tivermos e somarmos suas áreas, mais próximos estaremos da área real para esse intervalo sob a curva? Essa abordagem é o que chamamos de **Somas de Reimann**. Espere um minuto, isso é um limite! À medida que nos aproximamos do número de retângulos ao infinito, isso não convergiria para a área real sob a curva?

Vamos executar uma aproximação usando 10.000 retângulos, sem gráfico. Vamos colocar isso nesta função que aproxima aproximadamente usando esta abordagem das Somas de Reimann.

In [ ]:
def integral_aproximada(a, b, n, f):
    delta_x = (b - a) / n
    total_sum = 0

    for i in range(1, n + 1):
        leftpoint = a + delta_x * (i - 1)
        total_sum += f(leftpoint)

    return total_sum * delta_x

area = integral_aproximada(a=0, b=1, n=10_000, f=f)

print(area) # 1.6665666699999973

Vamos ser mais precisos. Vamos usar o SymPy, aproximar-nos de um número infinito de retângulos e somá-los. Com certeza, obtemos $ 5 / 3 $.

In [ ]:
from sympy import *

# Declara variáveis ​​para SymPy
x, i, n = symbols('x i n')

# Declara função e intervalo
f = 2*x**2 + 1
lower, upper = 0, 1

# Calcula a largura e a altura de cada retângulo no índice "i"
delta_x = ((upper - lower) / n)
x_i = (lower + delta_x * i)
fx_i = f.subs(x, x_i)

# Itera todos os "n" retângulos e soma suas áreas
n_rectangles = Sum(delta_x * fx_i, (i, 1, n)).doit()

# Calcula a área aproximando o número
# de retângulos "n" ao infinito
area = limit(n_rectangles, n, oo)

area # prints 5/3

Felizmente, assim como com as derivadas, não precisamos fazer todo esse trabalho de limite. Em vez disso, podemos usar a função `integrate()`, como mostramos anteriormente. Ela cuidará de todo o trabalho de limite nos bastidores.

In [ ]:
from sympy import * 

x = symbols('x')
f = 2*x**2 + 1

integrate(f, (x,0,1))

Observe que, quando fornecemos um intervalo, como mostrado no exemplo anterior, o chamamos de **integral definida**. Denotamos assim:

$
\Large \int_{0}^{1} (2x^2 + 1) = \frac{5}{3}
$


Se quiséssemos generalizar a função integral sem nenhum intervalo específico, a chamaríamos de **integral indefinida**.

$
\Large \int (2x^2 + 1) = \frac{2 x^{3}}{3} + x
$

Podemos calcular uma integral indefinida assim no SymPy.

In [ ]:
from sympy import * 

x = symbols('x')
f = 2*x**2 + 1

integrate(f,x)

Lembre-se de que a integral indefinida captura a área de menos infinito até o valor dado de $ x $. Portanto, se quiséssemos calcular a área de $ x = 0 $ a $ x = 1 $, substituiríamos ambos os valores na integral indefinida e subtraíríamos o último do primeiro.

$ 
\Large \int_{0}^{1} (2x^2 + 1) = (\frac{2 (1)^{3}}{3} + 1) - (\frac{2 (0)^{3}}{3} + 0) = \frac{5}{3}
$ 

## Exemplo de Distribuição Normal

Aqui está um exemplo aplicado de integrais, bastante utilizado em estatística. Sempre que trabalhamos com distribuições de probabilidade contínuas, precisamos encontrar áreas sob a curva (a função densidade de probabilidade) para calcular probabilidades para um determinado intervalo de valores de $ x $. Normalmente, para calcular essas áreas, usamos uma função densidade cumulativa, que provaremos ser apenas uma integral.

Uma distribuição comum é uma distribuição normal, cuja função densidade de probabilidade é formulada por:



$
\Large f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2}
$

$ \mu $ é a média, $ \sigma $ é o desvio padrão e $ e $ é o número de Euler que aprendemos. O valor $ x $ é a variável de entrada. Isso produzirá a famosa curva de sino, que podemos plotar no SymPy abaixo. Para simplificar, vamos definir a média como $ 0 $ e o desvio padrão como $ 1 $, o que define uma distribuição normal padrão.

In [ ]:
from sympy import * 

x,u,s = symbols('x u s')
f = 1 / (s * sqrt(2*pi)) * E**(-1/2 * ((x - u)/s)**2)

# uma distribuição normal padrão tem uma
# média de 0 e um desvio padrão de 1
padrão_normal = f.subs([(u,0), (s, 1)])

plot(padrão_normal, xlim=(-4,4), ylim=(-0,.5))

Vamos calcular a área até $ x = 1 $, que está sombreada abaixo.

In [ ]:
x_array = np.linspace(-4, 1, 1000)
f_array = lambdify(x, padrão_normal)(x_array)

plot(padrão_normal, xlim=(-4,4), ylim=(-0,.5), fill={'x': x_array,'y1':f_array, 'alpha': .5})

Vamos calcular essa área até $ 1 $ usando a função `integrate()`. Novamente, usaremos a distribuição normal padrão, que tem $ \mu $ e $ \sigma $ substituídos por $ 0 $ e $ 1 $, respectivamente. Observe que as caudas em uma distribuição normal se estendem ao infinito e se aproximam do eixo x para sempre. Portanto, incluiremos infinito negativo como o início do intervalo e 1 como o fim do intervalo.

In [ ]:
integrate(padrão_normal,(x,-oo, 1)).evalf()

Portanto, esta área entre o infinito negativo e $ 1 $ resulta em uma área de $ 0,8413 $. Com certeza, se compararmos isso com o cálculo do SciPy usando a função de densidade cumulativa, obteremos quase exatamente a mesma resposta. Isso mostra, com certeza, que a função de densidade cumulativa `cdf()` é apenas uma operação de integração.

In [ ]:
from scipy.stats import norm 

norm.cdf(1, 0, 1) # x, mean, std

## Exercício

Para esta função:

$ 
f(x) = 5x^3 - 10 
$ 

Calcule a área de $ x = 2 $ a $ x = 3 $ substituindo o ponto de interrogação abaixo.

In [ ]:
from sympy import * 

x = symbols('x')
f = 5*x**3 - 10 

?

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

Você deve encontrar uma área de $ \frac{258}{4} $ usando a função `integrate()`.

In [ ]:
from sympy import * 

x = symbols('x')
f = 5*x**3 - 10 

integrate(f, (x,2,3))